In [ ]:
import torch
import torch.nn as nn
from emonet.models import EmoNet

In [ ]:
""" If we want to transfer learn model for difference number of emotions, change n_expression or n_reg """
state_dict_path = f"pretrained/emonet_8.pth"
n_expression = 10 # number of discrete classes
n_reg = 3 # number of dimensional classes

In [3]:
# load pretrained weights (state dict)
state_dict: dict[str, torch.Tensor]
state_dict = torch.load(state_dict_path, map_location="cpu")
state_dict = { k.replace("module.", ""): v for k, v in state_dict.items() }

In [ ]:
# print layers in pretrained weights (tensors)
total_numbers = 0
for idx, (name, tensor) in enumerate(state_dict.items()):
    numbers = 1
    print('{:>3} : {:>20}   {:<45}'.format(idx, str(tuple(tensor.shape)), name))
    for x in tensor.shape:
        numbers *= x
    total_numbers += numbers

print('total_numbers: {:_}'.format(total_numbers))

In [ ]:
# replace final layer with linear layer
state_dict['emo_fc_2.3.weight'] = torch.randn(n_expression+n_reg, 128)
state_dict['emo_fc_2.3.bias'] =   torch.randn(n_expression+n_reg)

In [ ]:
# load model with 
model = EmoNet(n_expression=n_expression, n_reg=n_reg).to("cuda:0")
model.load_state_dict(state_dict, strict=False)
model.eval() # set to inference mode

In [6]:
# freeze all layers except except for final layer
for name, param in model.named_parameters():
    if 'emo_fc_2' not in name and 'conv1x1_input_emo_2' not in name:
        param.requires_grad = False

In [ ]:
# Custom dataset class
from torch.utils.data import Dataset
import torchvision.transforms as transforms

""" Overload __init__(), __len__() and __getitem__() methods """
class SomeDataset(Dataset):

    def __init__(self, path: str, train: bool, transform: transforms.Compose|None):
        self.path = path
        self.train = train
        self.transform = transform
        ...
        
    def __len__(self):
        ...
        return 1

    def __getitem__(self, index):

        # if self.transform is not None:
        #     image = self.transform(image)
        
        ...

In [ ]:
# set up datasets and dataloaders
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([transforms.ToTensor()])

train_set = SomeDataset('path', train=True, transform=transform)
validate_set = SomeDataset('path', train=False, transform=transform)
# test_set = SomeDataset('path', train=False, transform=transform)

batch_size=16
train_loader =    DataLoader(train_set, batch_size=batch_size, shuffle=True)
validate_loader = DataLoader(validate_set, batch_size=batch_size, shuffle=True)
# test_loader =    DataLoader(test_set, batch_size=batch_size, shuffle=True)

In [11]:
# initialize loss function and optimizer
import torch.optim as optim

loss_fn = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# copied from: https://docs.pytorch.org/tutorials/beginner/introyt/trainingyt.html
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    for i, (inputs, labels) in enumerate(train_loader):

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(train_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

In [ ]:
# TRAIN
# copied from:  https://docs.pytorch.org/tutorials/beginner/introyt/trainingyt.html
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)


    running_vloss = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(validate_loader):
            vinputs, vlabels = vdata
            voutputs = model(vinputs)
            vloss = loss_fn(voutputs, vlabels)
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

In [13]:
# save model weights
torch.save(model.state_dict(), "weights/model_weights.pth")